Project Report

Instructions

This map is a very simple guide to making a distribution map of an organism.This project uses an open online database (In this map, we use data from gbif).

The project aims to filter and gather organism occurrence data to build a data frame, eventually, converting it to a numpy array and adding it to the basemap. It involves two main components, the API query section and the plotting section.

First, we need to think an organism and a continent of interest. (as a demonstration, I use platypus distribution in Oceania.)

In [ ]:
#platypus distribution in Oceania.
name = 'Ornithorhynchus anatinus'
region ='Oceania'

For the preparation, we need to set up the basic URL of the database and other parameters. 

In [ ]:
#Set up
#database API URL
data_url = "https://api.gbif.org/v1/occurrence/search?"
#Page parameter 'offset' indicate position and page, initial value set to 0.
offset = 0
#Page parameter 'limit' means the number of data present in a single page, max limit for gbif is 300.
limit = 300
#endofRecords indicate if the current page is the last page, when endofRcords= True, we can stop requesting. Set to false to strat with.
endOfRecords = False
#Create a empty dataframe for our data.
df_main = pd.DataFrame()

Next, use the obtain_spKey function to get the speciesKey for that organism, so I only get spatial occurrence data for this species, avoiding relevant but unwanted data.

In [ ]:
#Obtain the speciesKey for platypus, sciencific name 'Ornithorhynchus anatinus'.
spKey=obtain_spKey(data_url, name)

Assemble all parameters we need to make the ultimate URL to request the data. However, the returning data may be too much to fit in a single page (exceed the limit=300), often, we have multiple pages of results in response. Here we have the endofRecord==false loop function to get all the pages until the last. (whenever move to next page, offset would become the last offset + limit.

i.e.
One first page, offset=0 (the starting position), limit=300
One second page, offset=0+limit= 0+300 (the last page contain 300 data, now we are on a new page).
One third page, offset=300+limit=300+300 = 600 (now we are at position 600)
...

In [ ]:
params=(set_up_params(spKey, region, limit))
print(params)

In [ ]:
#We send a request to the gbif server, scan and load all the results into our data frame
#However, the data might be too much to fit in a single page (exceed the limit=300), often, we have multiple pages of results in response.
#Here we have the endofRecord==false loop function to get all the pages until the last.(whenever move to next page, offset would beocome last offset + limit.)
while endOfRecords == False:
    endOfRecords, df_main= server_scan(data_url, offset, params, df_main)
    offset += limit

Now we have a pandas dataframe df_main about matched occurrence data of platypus. Convert it to a csv file preparing for plotting.

In [ ]:
#Convert the dataframe format to csv for easier management.
get_csv(df_main)

In [ ]:
Get the corrdinate information we need

In [ ]:
#The results contain lots of information, but we are only interested in coordinates here.
df_corrdinate= df_main[['decimalLongitude', 'decimalLatitude']]
df_corrdinate.head

In [ ]:
Form a numpy array

In [ ]:
#To plot, convert the data to numpy array
numpy_array = df_corrdinate.to_numpy()
print(numpy_array)
print(type(numpy_array))

Create a basemap dict

In [ ]:
map_tiles_dictionary = basemap_source()

Plot!

In [ ]:
map_tiles = map_tiles_dictionary[basemap_name]

fig = plt.figure(figsize=(12, 12), facecolor="none")

# Create a GeoAxes in the tile's projection.
# Without it, the lon and lat would not be recogonised.
ax = plt.axes(projection=map_tiles.crs)

# Set the size of the map
ax.set_extent(map_extent)

# Add the on-demand image and resolution
ax.add_image(map_tiles, 10)

#prepartion on load your point data
#I transposed the numpy_arrray to make it suitable for plotting
oalon, oalat= numpy_array.T

#plot point data of organism distribution 
plt.scatter(oalon, oalat, linewidth=0, c='red', transform=ccrs.PlateCarree(), alpha=0.5, zorder=2)

List of dependencies

In [ ]:
#API query
import requests
import json
import pandas as pd
import numpy as np
from urllib.parse import urlencode

#plotting
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt

For the API requesting part, we need the requests package to send a request to the server, JSON package to format the data, pandas packages to create a data frame, NumPy package to form a plottable array and urlencode to rewrite information to format used for URL.
For the plotting part, cartopy package is useful to add base map, axes and data point, and matplotlib.pyplot package actucally does the plotting.

Describe testing

The testing code is trivial, which only checks if the code returns the expected data type and the number of elements.
The information can be double-checked by accessing the gbif database website.

Limitations / Future Improvments

The main limitation would be the overall framework best for gbif database and may not be applicable to other databases using different formatting and structure.
The second limitation is the testing, which is too trial and vague. It does not test for some critical parts which may break the code.